## project_nutrients_review_negative

In [1]:
import numpy as np

In [43]:
# project_nutrients_review_negative.csv
import pandas as pd
# df_review_negative = pd.read_csv('../datasets/project_nutrients_review_negative.csv')
df_review_negative = pd.read_csv('../datasets/reviews_data/redrop_all_preprocess_positive.csv')

In [44]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84215 entries, 0 to 84214
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0.1         84215 non-null  int64  
 1   Unnamed: 0           84215 non-null  int64  
 2   product_name         84215 non-null  object 
 3   review_date          84215 non-null  object 
 4   review_content       84215 non-null  object 
 5   review_star          84215 non-null  float64
 6   review_writer        84215 non-null  object 
 7   reveiw_replaced      84215 non-null  object 
 8   stopremoved_reviews  84215 non-null  object 
 9   okt_reviews          84215 non-null  object 
 10  replaced_review      84215 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 7.1+ MB


In [45]:
df_review_negative = df_review_negative.dropna()

### LDA 분석

##### 문장 벡터화

In [46]:
from gensim.corpora import Dictionary

In [47]:
# df_review_negative dictionary
dictionary = Dictionary(df_review_negative[['replaced_review']].values)
dictionary

In [48]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

[(3, 1)]

In [49]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [50]:
df_review_negative['문장벡터화'] = corpus_list

In [51]:
df_review_negative.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'product_name', 'review_date',
       'review_content', 'review_star', 'review_writer', 'reveiw_replaced',
       'stopremoved_reviews', 'okt_reviews', 'replaced_review', '문장벡터화'],
      dtype='object')

##### 토픽 잡기
- review_negative.csv 토픽 분석
- 토픽 수 9

In [52]:
from gensim.models import LdaModel

In [53]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=9)

In [54]:
lda_model.print_topics(num_words=4)

[(0, '0.005*"성능 기한 가성" + 0.003*"가격 좋다" + 0.003*"성능 기한 선물" + 0.002*"성능 기한 포장"'),
 (1, '0.026*"배송" + 0.014*"가격 효과" + 0.010*"효과" + 0.006*"성능 기한 늘 먹다 제품"'),
 (2,
  '0.031*"성능 기한 제품" + 0.013*"성능 기한 복용" + 0.008*"성능 기한 항상" + 0.006*"성능 기한 가격 대비"'),
 (3, '0.614*"성능 기한" + 0.004*"포장 상태 효과" + 0.003*"구매" + 0.002*"성능 기한 항상 먹다 제품"'),
 (4, '0.107*"기한" + 0.053*"포장 상태" + 0.030*"성능 기한 배송" + 0.007*"성능 기한 주문"'),
 (5,
  '0.041*"성능 기한 효과" + 0.010*"가격 배송" + 0.008*"성능 기한 부모님" + 0.007*"성능 기한 배송 제품"'),
 (6,
  '0.266*"성능" + 0.027*"성능 기한 구매" + 0.012*"성능 기한 빠르다 배송" + 0.002*"성능 기한 구매 효과"'),
 (7,
  '0.339*"기한 포장 상태" + 0.012*"성능 기한 좋다 제품" + 0.006*"성능 기한 구입" + 0.004*"성능 기한 좋다"'),
 (8,
  '0.455*"가격" + 0.002*"성능 기한 유통 배송" + 0.001*"성능 기한 효과 복용" + 0.001*"성능 배송 제품"')]

##### 최적에 토픽 단어 수

In [55]:
sentences = df_review_negative['replaced_review']
sentences

0        성능 기한 감마 리놀렌산 여성 다해 초록색 비교 브리 결정 함량 제조 과정 마음 구...
1        기한 포장 상태 두번째 구입 월경 증후군 효과 프로 정도 영향 다시 한번 주문 알약...
2        기한 포장 상태 유트브 구매 피부 염증 판매 답글 고객 브리 제품 건강 많다 도움 ...
3        기한 포장 상태 효과 판매 답글 고객 브리 제품 구매 건강 많다 도움 문의사항 있다...
4        성능 기한 케이스 달라 확인 배송 총알 알약 크기 작다 가로 길이 정도 이구 리뷰 ...
                               ...                        
84210              성능 지금 아르기닌 함량 그래서 피로 회복 되다 느낌 아침 상큼 스타일
84211                                                성능 기한
84212                                                성능 기한
84213                                             성능 지인 선물
84214                                                   성능
Name: replaced_review, Length: 84215, dtype: object

In [56]:
type(sentences)

pandas.core.series.Series

In [57]:
# 각 문장을 공백으로 나누어 리스트에 저장
tokenized_sentences = [sentence.split() for sentence in sentences]

In [58]:
# 결과 출력
morphs_list = list()
for tokens in tokenized_sentences:
    morphs_list.append(tokens)
morphs_list

[['성능',
  '기한',
  '감마',
  '리놀렌산',
  '여성',
  '다해',
  '초록색',
  '비교',
  '브리',
  '결정',
  '함량',
  '제조',
  '과정',
  '마음',
  '구매',
  '효과',
  '알약',
  '개봉시',
  '주의',
  '껍질',
  '제거',
  '습관',
  '하나',
  '얼른',
  '양제',
  '실선',
  '가위',
  '가요'],
 ['기한',
  '포장',
  '상태',
  '두번째',
  '구입',
  '월경',
  '증후군',
  '효과',
  '프로',
  '정도',
  '영향',
  '다시',
  '한번',
  '주문',
  '알약',
  '크기',
  '편입',
  '판매',
  '답글',
  '고객',
  '브리',
  '제품',
  '구매',
  '건강',
  '많다',
  '도움',
  '문의사항',
  '있다',
  '언제',
  '연락'],
 ['기한',
  '포장',
  '상태',
  '유트브',
  '구매',
  '피부',
  '염증',
  '판매',
  '답글',
  '고객',
  '브리',
  '제품',
  '건강',
  '많다',
  '도움',
  '문의사항',
  '있다',
  '언제',
  '연락'],
 ['기한',
  '포장',
  '상태',
  '효과',
  '판매',
  '답글',
  '고객',
  '브리',
  '제품',
  '구매',
  '건강',
  '많다',
  '도움',
  '문의사항',
  '있다',
  '언제',
  '연락'],
 ['성능',
  '기한',
  '케이스',
  '달라',
  '확인',
  '배송',
  '총알',
  '알약',
  '크기',
  '작다',
  '가로',
  '길이',
  '정도',
  '이구',
  '리뷰',
  '보고',
  '구매',
  '호주',
  '영양',
  '제로',
  '원료',
  '성분',
  '달맞이꽃',
  '자유',
  '보라',
  '양제',
  '캐나다',
  '산이',
  '

In [59]:
preprocessed_sentences = morphs_list

In [60]:
dictionary = Dictionary(preprocessed_sentences) # fix in like sklean
dictionary

In [61]:
# 일관성 점수
from gensim.models.coherencemodel import CoherenceModel
coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
coherenceModel.get_coherence()

KeyError: 50061

In [ ]:
# 혼잡도 점수
lda_model.log_perplexity(corpus_list)

In [24]:
start_topic = 2
end_topic = 10
coherence_scores = list()
perplexity_scores = list()
for topic_number in range(start_topic, end_topic+1):
    lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=topic_number) # fix
    coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
    coherence_scores.append(coherenceModel.get_coherence()) # 일관성 점수
    perplexity_scores.append(lda_model.log_perplexity(corpus_list))

In [ ]:
import pandas as pd
scores = pd.DataFrame([coherence_scores,  perplexity_scores]).T
scores

In [26]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(coherence_scores)
plt.show()
sns.lineplot(perplexity_scores)
plt.show()

##### LDA 시각화(review_negative)

In [28]:
# %pip install pyLDAvis

In [29]:
import pyLDAvis.gensim_models

In [30]:
pyLDAvis.enable_notebook() # 일반적인 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [31]:
pyLDAvis.display(result_visualized)

In [ ]:
# project_nutrients_review_negative_result_visualized.html
pyLDAvis.save_html(result_visualized, '../datasets/project_nutrients_review_negative_result_visualized.html')

## project_nutrients_review_positive

In [ ]:
import pandas as pd
df_review_positive = pd.read_csv('../datasets/project_nutrients_review_positive.csv')

In [33]:
df_review_positive.info()

In [35]:
df_review_positive = df_review_positive.dropna()

### LDA 분석

##### 문장 벡터화

In [36]:
# 'replaced_review' 열의 모든 값을 문자열로 변환
df_review_positive['replaced_review'] = df_review_positive['replaced_review'].astype(str)

In [ ]:
dictionary = Dictionary(df_review_positive[['replaced_review']].values)
dictionary

In [38]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_positive[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [39]:
df_review_positive['문장벡터화'] = corpus_list

In [ ]:
df_review_positive[:3]

##### 토픽 잡기(review_positive)
- review_positive.csv 토픽 분석
- 토픽 수 9 <<<<<<< 변경하기 

In [41]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=9)

In [ ]:
lda_model.print_topics(num_words=4)

##### 최적에 토픽 단어 수

In [ ]:
sentences = df_review_positive['replaced_review']
sentences

In [ ]:
type(sentences)

In [ ]:
# 각 문장을 공백으로 나누어 리스트에 저장
tokenized_sentences = [sentence.split() for sentence in sentences]

In [ ]:
# 결과 출력
morphs_list = list()
for tokens in tokenized_sentences:
    morphs_list.append(tokens)
morphs_list

In [ ]:
preprocessed_sentences = morphs_list

In [ ]:
dictionary = Dictionary(preprocessed_sentences) # fix in like sklean
dictionary

In [ ]:
# 일관성 점수
from gensim.models.coherencemodel import CoherenceModel
coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
coherenceModel.get_coherence()

In [ ]:
# 혼잡도 점수
lda_model.log_perplexity(corpus_list)

In [ ]:
start_topic = 2
end_topic = 10
coherence_scores = list()
perplexity_scores = list()
for topic_number in range(start_topic, end_topic+1):
    lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=topic_number) # fix
    coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
    coherence_scores.append(coherenceModel.get_coherence()) # 일관성 점수
    perplexity_scores.append(lda_model.log_perplexity(corpus_list))

In [ ]:
import pandas as pd
scores = pd.DataFrame([coherence_scores,  perplexity_scores]).T
scores

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(coherence_scores)
plt.show()
sns.lineplot(perplexity_scores)
plt.show()

##### LDA 시각화

In [43]:
pyLDAvis.enable_notebook() # 일반적인 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [44]:
pyLDAvis.display(result_visualized)

In [45]:
pyLDAvis.save_html(result_visualized, '../datasets/project_nutrients_review_positive_result_visualized.html')